# Машинное обучение, ФКН ВШЭ

# Практическое задание 7

## Общая информация

Дата выдачи: 11.05.2017

Срок сдачи: 23:59MSK 28.05.2017

### О задании

Практическое задание 7 посвящено задаче построения рекомендаций и различным подходам к её решению. В рамках данного задания вы:
 * опробуете различные подходы к построению рекомендаций в задаче рекомендаций фильмов;
 * оцените качество работы различных подходов на различных группах пользователей, изучите наличие известных недостатков рекомендательных систем;
 * познакомитесь со способами оценивания качества работы полученных рекомендательных систем, изучите их достоинства и недостатки.
 
### Оценивание и штрафы

Каждая из задач имеет определенную «стоимость» (указана в скобках около задачи). Максимально допустимая оценка за работу — 10 баллов.

Сдавать задание после указанного срока сдачи нельзя. При выставлении неполного балла за задание в связи с наличием ошибок на усмотрение проверяющего предусмотрена возможность исправить работу на указанных в ответном письме условиях.

Задание выполняется самостоятельно. «Похожие» решения считаются плагиатом и все задействованные студенты (в том числе те, у кого списали) не могут получить за него больше 0 баллов (подробнее о плагиате см. на странице курса). Если вы нашли решение какого-то из заданий (или его часть) в открытом источнике, необходимо указать ссылку на этот источник в отдельном блоке в конце Вашей работы (скорее всего вы будете не единственным, кто это нашел, поэтому чтобы исключить подозрение в плагиате, необходима ссылка на источник). 

Неэффективная реализация кода может негативно отразиться на оценке.


### Формат сдачи
Для сдачи задания переименуйте получившийся файл \*.ipynb в соответствии со следующим форматом: *HW7_Username.ipynb*, где *Username* — Ваша фамилия и инициалы на латинице (например, *HW7_IvanovII.ipynb*). Далее отправьте этот файл на hse.cs.ml+<номер группы>@gmail.com (например, hse.cs.ml+141@gmail.com для студентов группы БПМИ-141).

## Рекомендательные системы

В этой лабораторной работе будет рассмотрена задача предсказания оценки, которую пользователь поставит фильму. Особенность этой задачи в том, что объекты выборки описываются категориальными признаками, принимающими большое число значений (например: идентификатор пользователя, идентификатор фильма, тэги, киноперсоны).

Мы будем работать с датасетом [MovieLens + IMDb/Rotten Tomatoes](http://files.grouplens.org/datasets/hetrec2011/hetrec2011-movielens-2k-v2.zip) ([описание](http://files.grouplens.org/datasets/hetrec2011/hetrec2011-movielens-readme.txt)). Набор содержит данные о предпочтениях пользователей сервиса рекомендации кинофильмов [MovieLens](http://www.movielens.org/). Пользовательские оценки для фильмов принимают значения в интервале от 0.5 до 5.0, они записаны в файле *user_ratedmovies.dat* (а также в *user_ratedmovies-timestamps.dat*,  где для каждой оценки записана дата и время в формате timestamp), остальные файлы содержат дополнительную информацию о фильмах, которую можно использовать как признаки. Заметьте: кроме оценок (и тегов), про пользователя ничего не известно.

На основании этих данных необходимо построить модель, предсказывающую оценку пользователя фильму, который он еще не смотрел.

## Оценивание качества рекомендаций

Будем считать, что пользователю сервиса доступен блок рекомендаций, который может содержать рекомендации не более чем 5 фильмов.

Выберем некоторого пользователя $u$ и обозначим известные для него рейтинги за $R^u$. В качестве тестовых рейтингов $R^u_{test}$ для этого пользователя рассмотрим 5 рейтингов, поставленные последними по времени, в качестве валидационных $R^u_{val}$ — предпоследние 5 рейтингов. Остальные известные рейтинги этого пользователя будут составлять обучающую выборку $R^u_{train}$.

Для подбора гиперпараметров в рамках данного задания будем использовать валидационную выборку, предварительно обучив модель на обучающей выборке, а для финальной оценки качества — тестовую выборку, предварительно обучив модель на обучающей и валидационной выборках.

**1. (0.5 балла)** Загрузите данные и сформируйте 3 разреженные матрицы пользователи—фильмы для обучающих, валидационных и тестовых рейтингов пользователей соответственно, где в каждой ячейке стоит рейтинг, если он известен, или ноль, если неизвестен. Отметим, что в этом случае для валидационной и тестовой матриц в каждой строке должно быть ровно 5 ненулевых значений. Рассматривайте только тех пользователей, которые поставили оценки $\ge 11$ фильмам.

In [ ]:
# Your code here

Для измерения качества рекомендаций в этом задании будем использовать метрики RMSE@k и nDCG@k для $k=5$, описанные ниже.

#### RMSE@k

Поскольку нас интересуют лишь фильмы, попавшие в блок рекомендаций, качество работы модели можно оценивать при помощи RMSE на $k$ фильмах с наибольшим предсказанным рейтингом, где $k$ — размер блока рекомендаций. Отсортируем предсказанные моделью рейтинги $\hat{r}_{ui}$ в порядке убывания и обозначим $i$-ый элемент в полученной последовательности за $\hat{r}_{u(i)},$ а соответствующее этому фильму истинное значение рейтинга — за $r_{u(i)}$. Тогда RMSE@k вычисляется следующим образом:

$$ \text{RMSE@k}(u) = \sqrt{ \frac{1}{k} \sum_{i=1}^k (r_{u(i)} - \hat{r}_{u(i)})^2 },$$
$$ \text{RMSE@k} = \frac{1}{|U|} \sum_{u \in U} \text{RMSE@k}(u),$$
где $U$ — множество пользователей. При вычислении данной метрики все неизвестные оценки будем полагать равными 0.

#### nDCG@k

Для оценки качества рекомендаций также можно использовать метрику качества ранжирования. Для этого для каждого пользователя $u$ предскажем оценку для всех фильмов из $R^u_{test}$ и отсортируем эти фильмы по убыванию предсказанного рейтинга. Ожидается, что хороший алгоритм должен выдать релевантные фильмы вверху списка.

Как и ранее, отсортируем предсказанные моделью рейтинги $\hat{r}_{ui}$ в порядке убывания и обозначим $i$-ый элемент в полученной последовательности за $\hat{r}_{u(i)},$ а соответствующее этому фильму истинное значение рейтинга — за $r_{u(i)}.$

Напомним, что

$$\text{DCG@k}(u) = \sum_{i=1}^k g(r_{u(i)}) d(i),$$
$$\text{nDCG@k}(u) = \frac{\text{DCG@k}(u)}{\max \text{DCG@k}(u)},$$
$$\text{nDCG@k} = \frac{1}{|U|} \sum_{u \in U} \text{nDCG@k}(u),$$
где $g(r)$ — функция полезности фильма, а  $d(i)$ — штраф за позицию.

В рамках данного практического задания положим $g(r) = 2^r-1, \, d(i) = \frac{1}{\log_2 (i+1)}.$ При вычислении данной метрики все неизвестные оценки будем полагать равными 0.

**2. (1 балл)** Реализуйте функции rmse_score и ndcg_score, вычисляющие значения описанных выше метрик. Каждая из функций в качестве параметров должна принимать:
 * y_true — матрицу тестовых рейтингов (сформированную аналогично матрице тестовых рейтингов из предыдущего пункта; функция должна корректно работать и для разреженных, и для плотных матриц);
 * y_predicted — матрицу предсказаний модели в аналогичном формате (функция должна корректно работать и для разреженных, и для плотных матриц);
 * k — параметр $k$ в определениях метрик.

In [ ]:
def rmse_score(y_true, y_predicted, k=5):
    # Your code here
    pass

def ndcg_score(y_true, y_predicted, k=5):
    # Your code here
    pass

**3. (0.5 балла)** Разделите индексы пользователей в матрицах из п.1 на 3 группы:
1. пользователи, оценившие менее 20 фильмов;
2. 50 случайных пользователей, оценивших фильмы, имеющие не более 5 оценок;
3. все остальные пользователи.

In [ ]:
# Your code here

**4. (0.5 балла)** Постройте рекомендации на основе **most popular** метода, при котором предсказанный рейтинг для некоторого фильма $i$ одинаков для всех пользователей и совпадает со средним значением рейтинга по всем пользователям, оценившим этот фильм, и вычислите значения метрик RMSE@5 и nDCG@5 для тестовой матрицы из п. 1.

In [ ]:
# Your code here

**5. (1 балл)** Постройте рекомендации на основе user-based коллаборативной фильтрации (подробно метод описан в [лекции 19](https://github.com/esokolov/ml-course-hse/blob/master/2016-spring/lecture-notes/lecture19-recommender.pdf), в качестве функции сходства используйте корреляцию Пирсона), при этом итоговое предсказание модели $\hat{r}_{ui}$ вычисляйте по следующей формуле:
$$\hat{r}_{ui} = \bar{r}_{u} + \frac{\sum_{v \in U(u)} w_{uv} (r_{vi} - \bar{r}_v)}{\sum_{v \in U(u)} w_{uv}},$$
где $\bar{r}_u$ — средний ретинг пользователя $u$, $w_{uv}$ — мера сходства пользователей $u$ и $v$, $U(u) = \{ v \in U \, | \, w_{uv} > \alpha\}$ — коллаборация пользователя $u$. Значение параметра $\alpha$ подберите на валидационной выборке.

Вычислите значения метрик RMSE@5 и nDCG@5 на тестовой выборке для пользователей из группы 3.

In [ ]:
# Your code here

**6. (0.5 балла)** Вычислите значения метрик RMSE@5 и nDCG@5 на тестовой выборке для пользователей из групп 1 и 2 (по отдельности) для рекомендаций, построенных в п. 5. Ощутимо ли различие со значениями метрик из п. 5? Как это можно объяснить?

In [ ]:
# Your code here

**Ответ:**

**7. (2 балла)** Постройте рекомендации при помощи модели со скрытыми переменными. Напомним, что в данном методе решается следующая оптимизационная задача:
$$\sum_{(u, i) \in R} (r_{ui} - \bar{r}_u - \bar{r}_i - \langle p_u, q_i\rangle)^2 + \lambda \sum_{u \in U} \| p_u\|^2 + \mu \sum_{i \in I} \| q_i\|^2 \to \min_{p_u, q_i}$$

Реализуйте построение рекомендаций при помощи любого из изученных методов оптимизации для данной задачи (SGD, ALS, HALS), подберите значения параметров регуляризации на валидационной выборке и постройте график зависимости метрик RMSE@5 и nDCG@5 от значения ранга разложения на валидационной выборке (рассмотрите как минимум 10 различных значений ранга разложения). Также разрешается использовать любые свободно распространяемые библиотеки при условии, что функционал совпадает с приведенным выше и используется один из приведенных методов оптимизации. Обращаем ваше внимание, что в оптимизационной задаче суммирование ведется лишь по известным элементам матрицы $R$.

In [ ]:
# Your code here

**8. (0.5 балла)** Постройте рекомендации с оптимальными значениями гиперпараметров из п. 7 и вычислите значения метрик RMSE@5 и nDCG@5 на тестовой выборке для пользователей из групп 1, 2 и 3 (по отдельности). Ощутимо ли различие между ними? Как это можно объяснить?

In [ ]:
# Your code here

**Ответ:**

**9. (0.5 балла)** Реализуйте построение рекомендаций путём разложения матрицы рейтингов с помощью [разреженного SVD](http://scikit-learn.org/stable/modules/generated/sklearn.decomposition.TruncatedSVD.html) (в предположении, что неизвестные рейтинги заменяются на нули) и последующего её восстановления и постройте график зависимости метрик RMSE@5 и nDCG@5 от значения ранга разложения на валидационной выборке (рассмотрите как минимум 10 различных значений ранга разложения). Постройте рекомендации с оптимальными значениями гиперпараметров  и вычислите значения метрик RMSE@5 и nDCG@5 на тестовой выборке для пользователей из групп 1, 2 и 3 (по отдельности). Как полученные значения отличаются от аналогичных из п. 8? Как это можно объяснить?

In [1]:
# Your code here

**Ответ:**

**10. (0.5 балла)** Сформируйте обучающие, валидационные и тестовые матрицы объект-признак для следующих наборов признаков:
 * id пользователя (бинарное кодирование) + id фильма (бинарное кодирование);
 * id пользователя (бинарное кодирование) + id фильма (бинарное кодирование) + средняя оценка пользователя;
 * id пользователя (бинарное кодирование) + id фильма (бинарное кодирование) + жанры фильма;
 * id пользователя (бинарное кодирование) + id фильма (бинарное кодирование) + жанры фильма + киноперсоны.

In [ ]:
# Your code here

**11. (1.5 балла)** Для каждой из выборок из предыдущего пункта постройте рекомендации при помощи факторизационных машин из библиотеки [LibFM](http://libfm.org) с использованием метода оптимизации ALS, подобрав оптимальную размерность разложения на валидационной выборке. Вычислите значения метрик RMSE@5 и nDCG@5 на тестовой выборке для оптимальных значений гиперпараметров. Какой набор признаков оказался самым удачным? Как это можно объяснить?
При выполнении данного задания вам также может пригодиться библиотека [pywFM](https://github.com/jfloff/pywFM), являющаяся python-обёрткой над LibFM.

In [ ]:
# Your code here

**Ответ:**

**12. (1 балл)** Приведите сравнение качества всех моделей, руководствуясь значениями описанных метрик. Какие из моделей оказались лучше других по каждой из метрик? Как это можно объяснить?

**Ответ**: